In [22]:
import requests
import pandas as pd
from pandas import DataFrame 
!pip install geopy
from geopy.extra.rate_limiter import RateLimiter


import matplotlib.pyplot as plt
import zipfile
import numpy as np
import io
import time
from pprint import pprint 
print("Import Successful")
#!wget https://www150.statcan.gc.ca/n1/tbl/csv/10100084-eng.zip
#!unzip 10100084-eng.zip

     |████████████████████████████████| 111 kB 13.3 MB/s eta 0:00:01
Import Successful


# Downloading the data

In [9]:



df = pd.read_csv("TorontoListings.csv")


# Data Cleaning

In [10]:
df.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.874600e+04,1.874600e+04,0.0,18746.000000,18746.000000,18746.000000,18746.000000,18746.000000,14788.000000,18746.000000,18746.000000
mean,2.673976e+07,1.087340e+08,NaN,43.680717,-79.397420,135.468473,10.786568,26.635602,1.214266,4.715246,125.690761
std,1.298332e+07,1.055992e+08,NaN,0.048263,0.064453,263.761517,36.913904,52.418576,1.597121,10.354866,136.844548
min,1.419000e+03,1.565000e+03,NaN,43.586710,-79.634800,11.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,1.634553e+07,2.049040e+07,NaN,43.645170,-79.426310,61.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,2.775978e+07,6.787450e+07,NaN,43.662815,-79.397350,99.000000,2.000000,6.000000,0.600000,1.000000,83.000000
75%,3.853432e+07,1.823192e+08,NaN,43.700153,-79.376860,150.000000,7.000000,27.000000,1.580000,3.000000,252.000000
max,4.578971e+07,3.711952e+08,NaN,43.836900,-79.127810,13137.000000,1125.000000,828.000000,15.440000,88.000000,365.000000


From above, We can see there is a column which is empty, Hence, We drop empty, NAN data from the set. 

In [11]:
df.head(300)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1419,Beautiful home in amazing area!,1565,Alexandra,NaN,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0
1,8077,Downtown Harbourfront Private Room,22795,Kathie & Larry,NaN,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365
2,12604,Seaton Village Parlour Bedroom,48239,Rona,NaN,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0
3,23691,Queen Bedroom close to downtown,93825,Yohan & Sarah,NaN,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240
4,26654,World Class downtown @CN Tower Theatre MTCC ga...,113345,Adela,NaN,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1294243,Enjoy Toronto Junction.,1025890,Amir,NaN,Junction Area,43.66711,-79.46453,Entire home/apt,80,365,5,2018-08-19,0.06,1,365
296,1299374,Beaches Butterfly Garden Suite,7065012,Jude,NaN,East End-Danforth,43.67828,-79.30960,Entire home/apt,93,7,62,2019-12-31,0.71,2,35
297,1301516,Luxury on Lake 2 Stories Penthouse,6169934,Fahad,NaN,Niagara,43.63531,-79.40055,Entire home/apt,643,30,8,2014-07-02,0.09,2,365
298,1301718,Uber Upper Beach 3 Bedroom Suite,377846,Nick & Nena,NaN,East End-Danforth,43.68345,-79.29434,Entire home/apt,107,7,21,2020-10-17,0.27,10,238


In [12]:

df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

Since id and host_name, both are giving the same information, We keep one of them and We will do the same thing for name and coordination(lattitude+longitude), We will concatenate them into coulumn location

In [26]:
# conveneint function to delay between geocoding calls
#geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
#df['location'] = df['ADDRESS'].apply(geocode)
# create longitude and altitude from location column (returns tuple)
#df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

AttributeError: 'float' object has no attribute 'point'

Now lets drop all rows with an Adjustment and then drop "Adjustments" and "Adjustments_cat". COORDINATE and VECTOR are now redundant too and only serve as a reference ID for the "Currency outside banks and chartered bank deposits" column. Lets drop COORDINATE.


In [27]:
df.drop(['neighbourhood_group'], 1, inplace = True)
df.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,location
0,1419,Beautiful home in amazing area!,1565,Alexandra,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0,-35.77834
1,8077,Downtown Harbourfront Private Room,22795,Kathie & Larry,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365,-35.73523
2,12604,Seaton Village Parlour Bedroom,48239,Rona,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0,-35.74874
3,23691,Queen Bedroom close to downtown,93825,Yohan & Sarah,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240,-35.75866
4,26654,World Class downtown @CN Tower Theatre MTCC ga...,113345,Adela,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295,-35.74410


Since VECTOR now uniquely maps to the description of the value. Lets make a dictionary out of the two columns and drop the "Currency outside banks and chartered bank deposits" column for brevity.

In [ ]:
mydict = df.set_index('VECTOR').to_dict()['Currency outside banks and chartered bank deposits']
df.drop('Currency outside banks and chartered bank deposits', axis = 1, inplace = True)

# Insights

Lets start by looking at the values available to us in our new dictionary and consider what insights are available to us:

In [ ]:
def getKeys():
    for (key, values) in mydict.items():
        print(key,values)

getKeys()

Tax sheltered deposits at character banks look like they have potential to be interesting. Lets look at that first.

In [ ]:
tax_msk = df[df['VECTOR'] == 'v37229']
tax_msk
#Tax_sheltered = df[column_msk]
#print(Tax_sheltered)

In [ ]:
plt.plot(tax_msk['REF_DATE'], tax_msk['VALUE'])
plt.xlabel("Date")
plt.ylabel("Dollars (MM)")
plt.title("The Amount of TAX sheltered accounts from 1968 to 1994", color="red" ,fontsize=10)

So the amount of money in tax sheltered accounts are clearly raising with time and appears to have some seasonal component. This is not a surprise as the RRSP contribution deadline among other factors are in March. With an extremely sharp increase through 1993. What happened in 1993? The only thing I find of significant importance is Brian Mulrooney's retirement announcement.

# Additional Plotting

Lets now plot all the series in the same plot and see what they look like.

In [ ]:
uniq_series = np.unique(df["VECTOR"])

In [ ]:
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = df.VECTOR == key
    x = df[key_mask].REF_DATE
    y = df[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

This is looking better, but there is some data from before 1976. What is it?

In [ ]:
df['REF_DATE'].value_counts()

It appears that there are three series in the set that began before 1976. Lets isolate them into a separate plot. First, we identify which vectors are available during that time period:

In [ ]:
veccounts = df[df['REF_DATE'].dt.year == 1969].VECTOR.value_counts()
print(veccounts)

There seem to be three vectors from 1969. Lets extract those into a list.

In [ ]:
veclist = np.unique(df[df['REF_DATE'].dt.year == 1969].VECTOR)
print(veclist)

Lets check what these vectors mean using our dictionary:

In [ ]:
[mydict[vec] for vec in veclist]

The data captured before 1976 are only various forms of deposits between banks. Specifically, these function essentially like chequing accounts that bank A holds with another bank B. Effectively they are like very liquid cash equivalents held at another institution that.

It looks like they are nested subsets of each other. That is the money counted in 'Inter-bank demand deposits' is also contained in 'Inter-bank demand and notice deposits' and so on. Lets confirm by plotting these three.

In [ ]:
for vec in veclist:
    plt_msk = df['VECTOR'] == vec
    plt.plot(df[plt_msk].REF_DATE, df[plt_msk].VALUE, label = mydict[vec])
    plt.title("Inter-bank Demand Deposites Compare to Inter-bank Demand Deposits and Notice Deposites Before 1980")
    #print('Now plotting', mydict[vec])
    #time.sleep(2)
plt.show()

It turns out one of these plots are exactly the same as the other. Lets take a closer look:

In [ ]:
for vec in veclist:
    plt_msk = df['VECTOR'] == vec
    plt.plot(df[plt_msk].REF_DATE, df[plt_msk].VALUE, label = mydict[vec])
    print('Now plotting', mydict[vec])
    plt.show()

Notice that there are only two lines. This is because there are no notice deposits. Hence there are only effectively two plots here:

    'Inter-bank demand deposits', and 'Inter-bank demand and term deposits'.
    
With this in mind. We can now drop these from the dataset and focus on remaining data.

Lets make a new copy of the dataframe and drop the old data.

In [ ]:
df2 = df.copy()

for vec in veclist:
    tmp_ind = df2[df2['VECTOR'] == vec].index
    df2.drop(tmp_ind, axis = 0, inplace = True)



In [ ]:
uniq_series = np.unique(df2["VECTOR"])
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = df2.VECTOR == key
    x = df2[key_mask].REF_DATE
    y = df2[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

As we can see, now. The vectors corresponding to 'Inter-bank demand deposits and other adjustments to M1', 'Inter-bank notice deposits and other adjustments to M2', and 'Inter-bank term deposits and other adjustments to M3' are extensions of the previous three plots we looked at. In 1976, the Bank of Canada separated these time series so that each one only looked at one thing - demand deposits, notice deposits, and term deposits as opposed to amalgamating them.

Note that since in 1976, there are no notice deposits. We can extent the three above time series back to 1976 by taking the difference of 'Inter-bank demand, notice and term deposits' and 'Inter-bank demand deposits'. For now, let us simply drop the data. As these represent a separate category from the remaining series.

In [ ]:
df3 = df2.copy()

iblist = ['v37226', 'v37227', 'v37228']

for vec in iblist:
    tmp_ind = df3[df3['VECTOR'] == vec].index
    df3.drop(tmp_ind, axis = 0, inplace = True)
    
uniq_series = np.unique(df3["VECTOR"])
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = df3.VECTOR == key
    x = df3[key_mask].REF_DATE
    y = df3[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

Next, let us group together all the physical currency such as coins and notes. i.e., lets remove the fiat currency from the data frame.

In [ ]:
getKeys()

From here, our list of fiat currencies is:

In [ ]:
fiatList = ['v37232', 'v37231', 'v37230']

In [ ]:
fiat_df = df3[df3['VECTOR'].isin(['v37232', 'v37231', 'v37230'])]
#fiat_df = df3[(df3['VECTOR'] == fiatList[0]) | (df3['VECTOR'] == fiatList[1]) | (df3['VECTOR'] == fiatList[2])]

In [ ]:
for vec in fiatList:
    tmp_ind = fiat_df[fiat_df['VECTOR'] == vec].index
    df3.drop(tmp_ind, axis = 0, inplace = True)
    
uniq_series = np.unique(fiat_df["VECTOR"])
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = fiat_df.VECTOR == key
    x = fiat_df[key_mask].REF_DATE
    y = fiat_df[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')
plt.title("Distribution Of The Different Currencies Over The Years")

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper right', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

Here, we can see that notes regularly get destroyed and new notes are added to circulation. Furthermore, we notice that coins did not dip in supply. This is not a surprise as coins can last more than 40 years. Furthermore, the Canadian silver 50 cent coin was already rarely used in the 60s with the final print run in large quantities ending in 1967. Furthermore, this plot does not extend far enough for the introduction of the toonie in 1996, or the removal of the penny in 2012.



Erik Chan  11:41 AM
I have a question about the python project if this is a good time






Erik Chan  11:49 AM
I'll just write it out instead:
I have been cleaning and looking at relationships in the data, but I keep finding that time and time again, the main take away I am getting from the data set is that there is a shocking amount of information that is inaccessible necessary to come to various key conclusions.





11:51
For example, a specific kind of deposit started to appear in 1975, and appeared until a legislative change occured in 1983 and data asks the reader to refer to a Bank of Canada Review document from March of 1983 which no longer exists because it is cut off from 1994.





11:52
Without those documents, it isn't very feasible to draw conclusions about some of the most interesting data points. Now my question is as follows: Is that in and of itself a conclusion I can use for the data set? (edited) 






Kristine  1:19 PM
Yes, absolutely!  I think that pointing out the limitations of the data would in fact be very useful.  For the purposes of Friday's exercise, we would also like you to be able to develop a visualization which helps to explain the data succinctly.  ANY graphical representation (think pie chart, graph, etc) counts here.  The use of visuals is a good tool in oral communication, but your bottom line - i.e. that the data is extremely limited - is a perfectly valid analysis.  Sorry for the slow reply.






Erik Chan  1:22 PM
That's very helpful. Thank you for the feedback






Kristine  1:51 PM
You’re welcome.  Feel free to repeat it.